In [1]:
#!pip install langchain

   ---------------------------------------- 0.0/302.9 kB ? eta -:--:--
   -------------- ------------------------- 112.6/302.9 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 302.9/302.9 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.28
    Uninstalling langchain-core-0.2.28:
      Successfully uninstalled langchain-core-0.2.28


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.20 requires langchain-core<0.3.0,>=0.2.26, but you have langchain-core 0.1.52 which is incompatible.


In [2]:
#!pip install streamlit

In [3]:
#!pip install PyPDF2

In [4]:
#!pip install langchain-openai

  Using cached langchain_core-0.2.28-py3-none-any.whl.metadata (6.2 kB)
Using cached langchain_core-0.2.28-py3-none-any.whl (379 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.52
    Uninstalling langchain-core-0.1.52:
      Successfully uninstalled langchain-core-0.1.52


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.17 requires langchain-core<0.2.0,>=0.1.48, but you have langchain-core 0.2.28 which is incompatible.
langchain-community 0.0.36 requires langchain-core<0.2.0,>=0.1.48, but you have langchain-core 0.2.28 which is incompatible.
langchain-text-splitters 0.0.1 requires langchain-core<0.2.0,>=0.1.28, but you have langchain-core 0.2.28 which is incompatible.


In [5]:
import os
from PyPDF2 import PdfReader
import streamlit as st
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
api_key="sk"

def process_text(text): 
#CharacterTextSplitter를 사용하여 텍스트를 청크로 분할
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)

    #임베딩 처리(벡터 변환), 임베딩은 OpenAI 모델을 사용합니다.
    embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", api_key=api_key)
    documents = FAISS.from_texts(chunks, embeddings)
    return documents

def main():  #streamlit을 이용한 웹사이트 생성
    st.title("📄PDF 요약하기")
    st.divider()

    pdf = st.file_uploader('PDF파일을 업로드해주세요', type='pdf')

    if pdf is not None:
        pdf_reader = PdfReader(pdf)
        text = ""   # 텍스트 변수에 PDF 내용을 저장
        for page in pdf_reader.pages:
            text += page.extract_text()

        documents = process_text(text)
        query = "업로드된 PDF 파일의 내용을 약 3~5문장으로 요약해주세요."  # LLM에 PDF파일 요약 요청

        if query:
            docs = documents.similarity_search(query)
            llm = ChatOpenAI(model="gpt-3.5-turbo-16k", api_key=api_key, temperature=0.1)
            chain = load_qa_chain(llm, chain_type='stuff')

            with get_openai_callback() as cost:
                response = chain.run(input_documents=docs, question=query)
                print(cost)

            st.subheader('--요약 결과--:')
            st.write(response)

if __name__ == '__main__':
    main()


2024-08-07 16:15:30.841 
  command:

    streamlit run C:\Users\JYSEO\anaconda3\envs\openai\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
